# Task 2: RRID Linking model with SciBERT

## 1.1 Using Colab for GPU Training

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device. 

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


## 1.2 Installing the Hugging Face Library

In [3]:
!pip install transformers

## 2. Loading Data

### 2.1 Download

Mounted Drive

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Set path

In [5]:
folderPath = '/content/gdrive/My Drive/antibody-watch/antibody-rrid/'
dataPath = '/content/gdrive/My Drive/antibody-watch/antibody-rrid/data/'

### 2.2 Parse

In [6]:
import pandas as pd
import re
import numpy as np

df = pd.read_csv(dataPath+'all-pair.csv')
df = df.loc[df['RRID GT'] != '-' ]
df.head()

,id,PMID,RRID Snippet_rdw_x,SNIPPET_3192antibody_x,RRID GT
0,0,23390418,The four Panx1 antibodies used in this study w...,The Diatheva and Dahl antibodies were two of t...,Yes
1,1,23390418,(2) chicken anti-Panx1 (obtained from Gerhard ...,The Diatheva and Dahl antibodies were two of t...,Yes
2,2,23390418,(3) rabbit anti-Panx1 (denoted as Rb57; NIF an...,The Diatheva and Dahl antibodies were two of t...,No
3,3,23390418,and (4) mouse anti-Panx1 (denoted as Mo503; NI...,The Diatheva and Dahl antibodies were two of t...,No
4,4,23390418,The four Panx1 antibodies used in this study w...,"For example, in the study of Panx1 knockout mi...",Yes


### 2.3 Preprocess data

The two properties we actually care about are the the `sentence` and its `label`, which is referred to as the "acceptibility judgment" (0=No, 1=Yes).  
Since URL is very long, we replace it with `'URL'`.

In [7]:
def preprocess_sentence(w):
    w = str(w).lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replace urls
    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                    .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
                    re.MULTILINE|re.UNICODE)
    w = re_url.sub("URL", w)

    # # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    return w

In [8]:
def preprocess_label(l):
    if l == 'Yes':
         return 1
    else:
         return 0

Let's preprocess our data

In [9]:
text_a = [preprocess_sentence(sent) for sent in df['SNIPPET_3192antibody'].values]
text_b = [preprocess_sentence(sent) for sent in df['RRID Snippet_rdw'].values]
labels = [preprocess_label(l) for l in df['RRID GT'].values]

## 3. Tokenization & Input Formatting

### 3.1 BERT Tokenizer

To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary.

The tokenization must be performed by the tokenizer included with BERT--the below cell will download this for us. We'll be using the "uncased" version here.

In [10]:
from transformers import AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True)

Loading BERT tokenizer...


### 3.2 Sentences to IDs

The `tokenizer.encode` function combines multiple steps for us:
1. Split the sentence into tokens.
2. Add the special `[CLS]` and `[SEP]` tokens.
3. Map the tokens to their IDs.

Oddly, this function can perform truncating for us, but doesn't handle padding. 

In [11]:
# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
input_ids = []
for i in range(len(text_a)):
    t = ['[CLS]']
    token = tokenizer.tokenize(text_a[i])[:127]
    t.extend(token)
    t.append('[SEP]')
    token = tokenizer.tokenize(text_b[i])[:126]
    t.extend(token)
    t.append('[SEP]')
    input_ids.append(t)
print(len(input_ids))

8130


In [12]:
print ("Tokenize the first sentence:")
print (input_ids[0])

Tokenize the first sentence:
['[CLS]', 'the', 'di', '##athe', '##va', 'and', 'dahl', 'antibodies', 'were', 'two', 'of', 'the', 'five', 'the', 'authors', 'claim', 'to', 'be', 'non', 'specific', '.', 'it', 'has', 'always', 'been', 'the', 'case', 'that', 'specificity', 'of', 'antibodies', ',', 'particularly', 'when', 'used', 'on', 'intact', 'tissues', ',', 'is', 'hard', 'if', 'not', 'impossible', 'to', 'prove', '.', 'by', 'these', 'criteria', ',', 'the', 'results', 'obtained', 'in', 'western', 'blots', 'in', 'the', 'present', 'study', 'should', 'cause', 'us', 'to', 'discard', 'our', 'findings', '.', '[SEP]', 'the', 'four', 'pan', '##x', 'antibodies', 'used', 'in', 'this', 'study', 'were', 'a', 'commercial', 'chicken', 'anti', 'pan', '##x', 'di', '##athe', '##va', 'ant', ',', 'denoted', 'as', 'ckd', '##ia', ',', 'di', '##athe', '##va', ',', 'fan', '##o', ',', 'italy', 'nif', 'antibody', 'registry', 'number', 'ab', ',', '[SEP]']


In [13]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in input_ids]

### 3.3 Padding & Truncating

Pad and truncate our sequences so that they all have the same length, `MAX_LEN`.

First, what's the maximum sentence length in our dataset?

In [14]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  256


Given that, let's choose MAX_LEN = 256 and apply the padding.

In [15]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 256 somewhat arbitrarily.
MAX_LEN = 256

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 256 values...

Padding token: "[PAD]", ID: 0

Done.


Using TensorFlow backend.


### 3.4 Attention Mask

The attention mask simply makes it explicit which tokens are actual words versus which are padding. 

The BERT vocabulary does not use the ID 0, so if a token ID is 0, then it's padding, and otherwise it's a real token.

In [16]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

## 4. Training

Define data spliting function

In [17]:
# Use KFold to split our data into train and validation sets for
# training
from sklearn.model_selection import KFold

# define KFold with n_splits=5 to make the dataset split test_size=1/n_splits
kf = KFold(n_splits=5, random_state=2018, shuffle=True)

def kfold_split(x, y, round_loop=1):
    count = 1
    for train_index, test_index in kf.split(x):
        x_train, x_test, y_train, y_test = [x[i] for i in train_index], [x[i] for i in test_index], [y[i] for i in train_index], [y[i] for i in test_index]
        if round_loop == count:
            break
        count += 1
    return x_train, x_test, y_train, y_test

Define helping functions

In [18]:
import numpy as np
from sklearn.metrics import classification_report, f1_score

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [19]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Start Training

In [20]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import random
import os

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

seed_val = 2018

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

max_acc = 0

for fid in range(5):
    print('>> Starting fold: ', fid)
    print('>>'*50)
    train_inputs, validation_inputs, train_labels, validation_labels = kfold_split(input_ids, labels, fid)
    train_masks, validation_masks, _, _ = kfold_split(attention_masks, input_ids, fid)

    # Convert all inputs and labels into torch tensors, the required datatype 
    # for our model.
    train_inputs = torch.tensor(train_inputs)
    validation_inputs = torch.tensor(validation_inputs)
    train_labels = torch.tensor(train_labels)
    validation_labels = torch.tensor(validation_labels)
    train_masks = torch.tensor(train_masks)
    validation_masks = torch.tensor(validation_masks)

    # Create the DataLoader for our training set.
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create the DataLoader for our validation set.
    validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
    
    # Load BertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = BertForSequenceClassification.from_pretrained(
        'allenai/scibert_scivocab_uncased', # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 2, # The number of output labels--2 for binary classification.
                        # You can increase this for multi-class tasks.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )

    # Tell pytorch to run this model on the GPU.
    model.cuda()

    # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
    # I believe the 'W' stands for 'Weight Decay fix"
    optimizer = AdamW(model.parameters(),
                    lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                    )

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs
    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
    
    # Store the average loss after each epoch so we can plot them.
    loss_values = []

    # For each epoch...
    for epoch_i in range(0, epochs):
        
        # ========================================
        #               Training
        # ========================================
        
        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
                
                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
            
            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple.
            loss = outputs[0]

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            
        
        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
            
        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        y_true = []
        y_pred = []
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            
            # Telling the model not to compute or store gradients, saving memory and
            # speeding up validation
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)
            
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = outputs[0]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            y_pred.extend(np.argmax(logits, axis=1).flatten())
            y_true.extend(label_ids.flatten())
            
            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy

            # Track the number of batches
            nb_eval_steps += 1

        # Report the final accuracy for this validation run.
        print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
        print("  Validation took: {:}".format(format_time(time.time() - t0)))

        if max_acc < eval_accuracy/nb_eval_steps:
            max_acc = eval_accuracy/nb_eval_steps
            
            output_dir = './model_save/'

            # Create output directory if needed
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            print("Saving model to %s" % output_dir)

            # Save a trained model, configuration and tokenizer using `save_pretrained()`.
            # They can then be reloaded using `from_pretrained()`
            model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
            model_to_save.save_pretrained(output_dir)
            tokenizer.save_pretrained(output_dir)

            # Good practice: save your training arguments together with the trained model
            # torch.save(args, os.path.join(output_dir, 'training_args.bin'))

        print(classification_report(y_true, y_pred))
        print("  F1 score: {}".format(f1_score(y_true, y_pred)))

    print("")
    print("Training complete!")

print("Done!!")

>> Starting fold:  0
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    407.    Elapsed: 0:00:33.
  Batch    80  of    407.    Elapsed: 0:01:06.
  Batch   120  of    407.    Elapsed: 0:01:39.
  Batch   160  of    407.    Elapsed: 0:02:13.
  Batch   200  of    407.    Elapsed: 0:02:46.
  Batch   240  of    407.    Elapsed: 0:03:20.
  Batch   280  of    407.    Elapsed: 0:03:53.
  Batch   320  of    407.    Elapsed: 0:04:26.
  Batch   360  of    407.    Elapsed: 0:05:00.
  Batch   400  of    407.    Elapsed: 0:05:33.

  Average training loss: 0.24
  Training epcoh took: 0:05:39

Running Validation...
  Accuracy: 0.94
  Validation took: 0:00:28
Saving model to ./model_save/
              precision    recall  f1-score   support

           0       0.98      0.95      0.97      1391
           1       0.76      0.89      0.82       235

    accuracy                           0.9

In [21]:
!ls -l --block-size=K ./model_save/
!cp -r ./model_save/ '/content/gdrive/My Drive/antibody-watch/antibody-rrid/'

total 429660K
-rw-r--r-- 1 root root      1K Jun 26 07:03 config.json
-rw-r--r-- 1 root root 429423K Jun 26 07:03 pytorch_model.bin
-rw-r--r-- 1 root root      1K Jun 26 07:03 special_tokens_map.json
-rw-r--r-- 1 root root      1K Jun 26 07:03 tokenizer_config.json
-rw-r--r-- 1 root root    223K Jun 26 07:03 vocab.txt


## For evaluation

In [23]:
output_dir = './model_save/'
# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
df = pd.read_csv(dataPath+'all-pair.csv')
df.head()

,id,PMID,RRID Snippet_rdw_x,SNIPPET_3192antibody_x,RRID GT
0,0,23390418,The four Panx1 antibodies used in this study w...,The Diatheva and Dahl antibodies were two of t...,Yes
1,1,23390418,(2) chicken anti-Panx1 (obtained from Gerhard ...,The Diatheva and Dahl antibodies were two of t...,Yes
2,2,23390418,(3) rabbit anti-Panx1 (denoted as Rb57; NIF an...,The Diatheva and Dahl antibodies were two of t...,No
3,3,23390418,and (4) mouse anti-Panx1 (denoted as Mo503; NI...,The Diatheva and Dahl antibodies were two of t...,No
4,4,23390418,The four Panx1 antibodies used in this study w...,"For example, in the study of Panx1 knockout mi...",Yes


In [26]:
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
text_a = [preprocess_sentence(sent) for sent in df['SNIPPET_3192antibody_x'].values]
text_b = [preprocess_sentence(sent) for sent in df['RRID Snippet_rdw_x'].values]
# labels = [preprocess_label(l) for l in df['Label: Yes/No'].values]

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for i in range(len(text_a)):
  t = ['[CLS]']
  token = tokenizer.tokenize(text_a[i])[:127]
  t.extend(token)
  t.append('[SEP]')
  token = tokenizer.tokenize(text_b[i])[:127]
  t.extend(token)
  input_ids.append(t)

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in input_ids]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)

# Set the batch size.  
batch_size = 16

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 25,130



In [27]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
# predictions , true_labels = [], []
predictions = []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  # b_input_ids, b_input_mask, b_labels = batch
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(np.argmax(logits, axis=1).flatten())
  # true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 25,130 test sentences...
    DONE.


### Save to File

In [28]:
def pred_to_label(pred):
  if pred == 1:
    return 'Yes'
  return 'No'

In [29]:
import csv
def PredictGenerator(data, prediction, filename='prediction.csv'):
    num_record = len(list(data.id.values))
    count = 0
    labels = []
    for i in prediction:
      for j in i:
        labels.append(j)
    with open(filename, mode='w') as csv_file:
      fieldnames = ['id', 'PMID','ground truth', 'prediction', 'SNIPPET_3192antibody', 'Snippet_rdw']
      writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

      writer.writeheader()
      for label in labels:
        if count < num_record:
          writer.writerow({
            'id': data.id.values[count],
            'PMID': data.PMID.values[count], 
            'ground truth': data['RRID GT'].values[count], 
            'prediction': pred_to_label(label), 
            'SNIPPET_3192antibody': data['SNIPPET_3192antibody_x'].values[count],
            'Snippet_rdw': data['RRID Snippet_rdw_x'].values[count]
          })
        count += 1

In [30]:
PredictGenerator(df, predictions, folderPath+'evaluation-rrid.csv')